## Question: How can we search for long-lived particle decays in the muon system?

### Signal event signature

The event signature for neutral long-lived particles is missing momentum aligned with a cluster of drift tube (DT) hits in the muon system, opposite to a high momentum jet. Example event display below (blue dots are DT hits, red arrow is the missing momentum, yellow cones are jets):

<img src="img/THeventDisplay.png" alt="img/THeventDisplay" width="500"/>

### DT clusters

Given DT hits in the muon system, clusters are formed to distinguish real particle decays from detector noise. Density-based spatial clustering of applications with noise (DBSCAN) algorithm is used to do the clustering.

DBSCAN algorithm takes all points and two parameters -- $N$, the minimum cluster size, and $\epsilon$, the clustering distance parameter -- and classifies every point as one of the following:

- Core point: at least $N$ points within the $\epsilon$ of the point
- Directly reachable point: not a core point but within $\epsilon$ of a core point
- Reachable point: there is a path from a core point through directly reachable points within $\epsilon$ ending at the point
- Noise: point does not satisfy any of the above categories

Starting with a core point, a cluster is formed by clustering the core point with all points reachable from it.

All the DT hits in an event are passed through the DBSCAN algorithm using their $(\eta, \phi)$ detector positions (implemented in C++ using [`DBSCAN.h`](https://github.com/sheplock/dtClusters/blob/main/DBSCAN.h) and [`DBSCAN.cc`](https://github.com/sheplock/dtClusters/blob/main/DBSCAN.cc)), and any successfully clustered group of hits is saved.

Example implementation using $N = 50$ and $\epsilon = 0.2$, where `DBSCAN::run()` and `DBSCAN::result()` are defined in `DBSCAN.cc`.
```C++
// Do DBSCAN Clustering
int min_point = 50;  // minimum number of hits to call it a cluster
float epsilon = 0.2; // cluster radius parameter
DBSCAN ds(min_point, epsilon, points); // points is a vector of Point types representing DT hits
ds.run();
ds.result();
```

Key functions for DBSCAN algorithm defined below.

```C++
// function called to run the DBSCAN algorithm
int DBSCAN::run()
{
    int clusterID = 1; // index used to classify which cluster each point belongs to
    vector<Point>::iterator iter; //iterator for DT hits
    for(iter = m_points.begin(); iter != m_points.end(); ++iter)
    {
        // check point hasn't been classified yet in another iteration of algorithm
        if ( iter->clusterID == UNCLASSIFIED ) 
        {
            // expandCluster() checks to see if point is core point, if so forms a cluster
            // all points in the cluster are labeled with clusterID
            // expandCluster() returns FAILURE if point can't seed new cluster
            // if point isn't failure (does seed a new cluster) then increase clusterID to start new cluster
            if ( expandCluster(*iter, clusterID) != FAILURE )
            {
                clusterID += 1;
            }
        }
    }
    // total number of clusters is clusterID-1 (initialized to 1 before any clusters made)
    nClusters = clusterID-1;
    return (clusterID-1);
}

int DBSCAN::expandCluster(Point point, int clusterID)
{
    //calculate all neighbors within distance parameter, including point itself
    vector<int> clusterSeeds = calculateCluster(point);

    // if fewer neighboring points than min cluster size, point is labeled as noise
    if ( clusterSeeds.size() < m_minPoints )
    {
        point.clusterID = NOISE;
        return FAILURE;
    }
    else // point is a core point
    {
        int index = 0, indexCorePoint = 0;
        vector<int>::iterator iterSeeds;
        // loop through neighbors of core points
        for( iterSeeds = clusterSeeds.begin(); iterSeeds != clusterSeeds.end(); ++iterSeeds)
        {
            // setting all the directly reachable points to the same clusterID
            m_points.at(*iterSeeds).clusterID = clusterID;
            // get the index of the core point itself
            if (m_points.at(*iterSeeds).eta == point.eta && m_points.at(*iterSeeds).phi == point.phi )
            {
                indexCorePoint = index;
            }
            // increment index for each time iterating through the loop
            ++index;
        }
        // remove the core point
        clusterSeeds.erase(clusterSeeds.begin()+indexCorePoint);
        // now clusterSeeds only contains directly reachable points, not core point itself
        // loop through the cluster again
        // if one of the directly reachable points is actually a core point, all points in the neighborhood of that point will be added to cluster (reachable points)
        for( vector<int>::size_type i = 0, n = clusterSeeds.size(); i < n; ++i )
        {
            // check if the neighbors themselves are core points
            vector<int> clusterNeighors = calculateCluster(m_points.at(clusterSeeds[i]));
            if ( clusterNeighors.size() >= m_minPoints ) // this point is a core point too
            {
                vector<int>::iterator iterNeighors;
                // iterate through the directly reachable points of the new core point
                for ( iterNeighors = clusterNeighors.begin(); iterNeighors != clusterNeighors.end(); ++iterNeighors )
                {
                    // new points that don't already belong to a cluster
                    if ( m_points.at(*iterNeighors).clusterID == UNCLASSIFIED || m_points.at(*iterNeighors).clusterID == NOISE )
                    {
                        // label points as belonging to this cluster
                        m_points.at(*iterNeighors).clusterID = clusterID;
                        
                        // if point hasn't been classified yet, add it to clusterSeeds
                        // then it will be checked for being a core point too
                        // (if noise, already know point isn't a core point from a previous iteration)
                        // if point is added, change n so the loop through all clusterSeeds contains new point
                        if ( m_points.at(*iterNeighors).clusterID == UNCLASSIFIED )
                        {
                            clusterSeeds.push_back(*iterNeighors);
                            n = clusterSeeds.size();
                        }
                        
                    }
                }
            }
        }

        return SUCCESS;
    }
}

// get index of all points in epsilon neighborhood of point
vector<int> DBSCAN::calculateCluster(Point point)
{
    int index = 0;
    vector<Point>::iterator iter;
    vector<int> clusterIndex;
    Point minimum;
    // iterate through all points, store index of points within epsilon
    for( iter = m_points.begin(); iter != m_points.end(); ++iter)
    {
        if ( calculateDistance(point, *iter) <= m_epsilon )
        {
            clusterIndex.push_back(index);
        }
        // index incremented for each loop of iterator
        index++;
    }
    return clusterIndex;
}

// distance used is the eta/phi 2D distance
inline double DBSCAN::calculateDistance( Point pointCore, Point pointTarget )
{
    return sqrt(pow(pointCore.eta - pointTarget.eta,2)+pow(deltaPhi(pointCore.phi, pointTarget.phi),2));

}
```

True noise in the detector is very unlikely to cause 50 hits, so noise will not typically be clustered. Signal decays cause clusters with high efficiency. From simulated signal events, the efficiency can be measured as a function of the particle decay location:

<img src="img/clusterReco_noVetoRadius_new.png" alt="img/clusterReco_noVetoRadius_new" width="500"/>

For hits that are clustered, signal decays tend to cause larger clusters than backgrounds. Even if noise is clustered, it tends to be just enough points to form a cluster of 50 hits, while the signal decays can cause many more hits:

<img src="img/clusterSize.png" alt="img/clusterSize" width="500"/>

### Punch-through prediction

A major source of background is jets in the inner parts of the detector that "punch-through" into the muon system and cause DT clusters. There are two handles to control this background: rejecting clusters matched to jets and rejecting clusters matched to 2+ MB1 hits (inner layer of DTs used for veto only, not for clustering).

These requirements remove vast majority of background clusters, however they can fail. The jet matching fails if the true jet energy is below 10 GeV (reconstruction does not work for this low energy). The MB1 veto fails due to gaps in the detector and detector inefficiency.

Therefore, clusters from jets below 10 GeV and with < 2 matched MB1 hits are expected to be signal region background. The 2D plane defined by these variables is shown below, where A2 is the signal region. CR2 clusters are rejected by the jet veto, A1 clusters are rejected by the MB1 veto, and CR1 clusters are rejected by both.

<img src="img/punchThroughABCDplane.png" alt="img/punchThroughABCDplane" width="500"/>

Since we cannot directly measure the background in the signal region (this must remain "blinded" until the final step of the analysis) we need a way to predict the background so that we know how many events to expect if there's no new signals.

To predict this punch-through background, the ratio of CR2/CR1 is measured as a function of jet energy. This allows us to construct a model so that we can predict the ratio of A2/A1. Then we can measure the background in A1 and calculate the expected background in A2.

The best fit for the CR2/(CR1+CR2) ratio is an exponential function plus a constant:

<img src="img/MB1VetoEfficiency_jetPt_MB2_expPlusConst_uncBand.png" alt="img/MB1VetoEfficiency_jetPt_MB2_expPlusConst_uncBand" width="500"/>

The full fitting/prediction is implemented in [`doPredictions.py`](https://github.com/sheplock/dtClusters/blob/main/doPredictions.py).

This fit allows us to extrapolate below 10 GeV, to get an expected ratio of A2/A1. Since the A1 yield is measurable, we can get an expected background contribution.

This method was validated in multiple validation regions to confirm that this approach gave a prediction that was consistent with the observed event yields in unblinded regions.

Relevant output from `doPredictions.py`:
(0.225, 0.02, 6.5, 2.1)

Veto efficiency = 0.225 $\pm$ 0.02

Background prediction = 6.5 $\pm$ 2.1

### Result:
Total expected background (including punch-through and other backgrounds) = 9.5 $\pm$ 1.9 events

Total signal region yield = 9 events

### Extra: Uncorrelated background prediction

To know if we've discovered a new particle, we must have a precise prediction for the expected background in the signal region. Since the signal region must remain "blinded" until the last step of the analysis, we need a method(s) to predict the background using measurements in other ("control") regions.

Noise background can be predicted using a "matrix" or "ABCD" method. For noise background, the two variables defining the plane drawn below are independent of each other:

<img src="img/ABCDplane.png" alt="img/ABCDplane" width="500"/>

Since they are independent, then the ratio of event yields is expected to be equal: $N_D / N_C = N_A / N_B$, or equivalently, $N_B / N_C = N_A / N_D$. This allows us to determine the background in the signal region (A) by measuring the yields in the other three regions: $N_A = N_B \times N_D / N_C$.

The only issue with the above method is that there is an additional background, from punch-through jets, where these varibles are not independent, so the prediction method is not valid for these events. This background is due to jets, which should be entirely contained in inner sections of the detector, escaping through into the outer muon system and causing DT hits. Just like signal decays, these clusters disproportionately populate region A, so the matrix method does not properly predict them.

The ABCD plane showing the population of these types of events is shown below:

<img src="img/ABCDevents_MB1CR.png" alt="img/ABCDevents_MB1CR" width="500"/>